In [1]:
print("nice")

nice


### global 

In [2]:
import os
import tensorflow as tf
import torch 
print(tf.__version__)
print(torch.__version__)

2.13.0
2.2.1+cpu


In [3]:
import numpy as np
print(np.__version__)
# setting random_state
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

1.24.3


### some libraries and functions 

In [4]:
# libraries
import sys, math 
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [5]:
# fix random_state
def fixRandomState(fixed_state: int=RANDOM_STATE):
  np.random.seed(fixed_state)
  tf.random.set_seed(fixed_state)
  torch.manual_seed(fixed_state)
  
# exception
def exception(requirement: bool, content):
  if(requirement == False): raise ValueError(content)
def catchException(ex: Exception):
  print(type(ex), ex.args)
  exception(False, ex)

# message
def mesVerbose(flag: bool, verbose, func_dir: str=""):
  if(flag == False): return
  print("__verbose__:", func_dir, verbose)
def mesWarning(note, func_dir: str=""):
  print("__warning__:", func_dir, str(note) + "@@@")

In [6]:
def over(val, name="") -> tuple:
  try: mesVerbose(True, (type(val), val.shape, str(sys.getsizeof(val)) + "Bytes"), name)
  except: mesVerbose(True, (type(val), "no shape", str(sys.getsizeof(val)) + "Bytes"), name)

In [7]:
class ModelHelp: 
  def plot(model, show_name: bool=False):
    # return keras.utils.plot_model(model,
    #   show_layer_names=show_name, show_layer_activations=True,
    #   show_shapes=True, show_dtype=True)
    pass 
  
  def debug(model):
    model.summary(show_trainable=True)
  
  def getOutputAbs(model): 
    return model.layers[-1].output 

### model architecture 

In [8]:
from keras import Sequential, Input
from keras.layers import (Conv2D, MaxPooling2D, Dense,
                                  BatchNormalization, LeakyReLU, Dropout, Activation,
                                  Flatten, Reshape)

INPUT_SHAPE = (224, 224, 3)

YOLO_BACKBONE_ARCHITECTURE = [(64, 7, 2, 'same'), 'M',
                                (192, 3, 1, 'same'), 'M',
                                (128, 1, 1, 'valid'),
                                [(128, 256), 1],
                                [(256, 512), 1], 'M',
                                [(256, 512), 4],
                                [(512, 1024), 1], 'M',
                                [(512, 1024), 2]]

##### blcoks

In [9]:
class ConvWithBatchNorm(Sequential):
  """Conv layer with batch norm and leaky relu"""
  
  def __init__(self, filters=64, kernel_size=3, strides=1, padding='same',
                activation=LeakyReLU(alpha=0.1), kernel_regularizer=None,
                name='conv', **kwargs):
    layers = [Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                        padding=padding, activation=None,
                        kernel_regularizer=kernel_regularizer, name=name, **kwargs)]
    # adding Layers together
    layers += [BatchNormalization(name=name + '_bn')]
    layers += [Activation(activation, name=name + '_act')]
    # inher customize
    super().__init__(layers=layers, name=name)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [10]:
class BottleNeckBlock(Sequential):
  """Block of 1x1 reduction layers followed by 3x3 conv. layer"""
  
  def __init__(self, filters, repetitions, name='bottleneck_block', **kwargs):
    filters_1x1 = filters[0]
    filters_3x3 = filters[1]
    model = []
    for i in range(repetitions):
        model += [ConvWithBatchNorm(filters=filters_1x1, kernel_size=1,
                                    strides=1, padding='valid',
                                    name='{}_conv_1x1_{}'.format(name, i + 1), **kwargs)]
        model += [ConvWithBatchNorm(filters=filters_3x3, kernel_size=3,
                                    strides=1, padding='same',
                                    name='{}_conv_3x3_{}'.format(name, i + 1), **kwargs)]
    ### inher customize
    super().__init__(layers=model, name=name)

##### YoloBackbone

In [11]:
class YoloBackbone(Sequential):
  """YOLO backbone extract feature from the input"""

  def __init__(self, input_shape=INPUT_SHAPE, backbone_config=YOLO_BACKBONE_ARCHITECTURE, name='YOLO_Backbone'):
    ### input_abstration 
    model = [keras.Input(shape=INPUT_SHAPE)]
    
    for i, config in enumerate(backbone_config):
        if type(config) == tuple:
          filters, kernel_size, strides, padding = config
          model += [ConvWithBatchNorm(filters, kernel_size, strides, padding,
                                      name='backbone_conv_{}'.format(i + 1))]
        elif type(config) == str:
          model += [MaxPooling2D(pool_size=2, strides=2, padding='same',
                                  name='backbone_max_pooling_{}'.format(i + 1))]
        elif type(config) == list:
          filters, repetition = config
          model += [BottleNeckBlock(filters, repetition,
                                    name='backbone_bottleneck_block_{}'.format(i + 1))]
    ### inher customize
    super().__init__(layers=model, name=name)

In [12]:
ModelHelp.debug(YoloBackbone())

Model: "YOLO_Backbone"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ backbone_conv_1             │ (None, 112, 112, 64)  │      9,728 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_max_pooling_2      │ (None, 56, 56, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_conv_3             │ (None, 56, 56, 192)   │    111,552 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_max_pooling_4      │ (None, 28, 28, 192)   │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_conv_5             │ (None, 28, 28, 128)   │     25,216 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_bottleneck_block_6 │ (None, 28, 28, 256)   │    313,216 │   Y   │
│ (BottleNeckBlock)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_bottleneck_block_7 │ (None, 28, 28, 512)   │  1,249,024 │   Y   │
│ (BottleNeckBlock)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_max_pooling_8      │ (None, 14, 14, 512)   │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_bottleneck_block_9 │ (None, 14, 14, 512)   │  5,258,240 │   Y   │
│ (BottleNeckBlock)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_bottleneck_block_… │ (None, 14, 14, 1024)  │  4,988,416 │   Y   │
│ (BottleNeckBlock)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_max_pooling_11     │ (None, 7, 7, 1024)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ backbone_bottleneck_block_… │ (None, 7, 7, 1024)    │ 10,501,120 │   Y   │
│ (BottleNeckBlock)           │                       │            │       │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 22,456,512 (85.66 MB)

 Trainable params: 22,438,080 (85.59 MB)

 Non-trainable params: 18,432 (72.00 KB)

##### YoloOutput 

In [13]:
class YoloOutput(Sequential):
  """YOLO last convolution and FC layers to produce prediction"""

  def __init__(self, grid_size=7, num_boxes=2, num_classes=20, name='YOLO_Output'):
    # input_abstraction 
    input_abs = ModelHelp.getOutputAbs(YoloBackbone()) 
    yolo_output = [keras.Input(shape=input_abs.shape[1:])]
    
    S, B, C = grid_size, num_boxes, num_classes
    yolo_output += [ConvWithBatchNorm(filters=1024, kernel_size=3, strides=1,
                                      padding='same', name='output_conv_1'),
                    ConvWithBatchNorm(filters=1024, kernel_size=3, strides=2,
                                      padding='same', name='output_conv_2'),
                    ConvWithBatchNorm(filters=1024, kernel_size=3, strides=1,
                                      padding='same', name='output_conv_3'),
                    ConvWithBatchNorm(filters=1024, kernel_size=3, strides=1,
                                      padding='same', name='output_conv_4'),
                    Flatten(),
                    Dense(units=4096, activation=LeakyReLU(alpha=0.1), name='output_fc_1'),
                    Dropout(rate=0.5, name='dropout_1'),
                    Dense(units=2048, activation=LeakyReLU(alpha=0.1), name='output_fc_2'),
                    Dropout(rate=0.5, name='dropout_2'),
                    Dense(units=1024, activation=LeakyReLU(alpha=0.1), name='output_fc_3'),
                    Dropout(rate=0.5, name='dropout_3'),
                    Dense(units=S * S * (B * 5 + C), name='prediction')]
    ### inher customize
    super().__init__(layers=yolo_output, name=name)

In [14]:
print(YoloBackbone())
print(ModelHelp.getOutputAbs(YoloBackbone())) 

<YoloBackbone name=YOLO_Backbone, built=True>
<KerasTensor shape=(None, 7, 7, 1024), dtype=float32, sparse=False, name=keras_tensor_360>


In [15]:
ModelHelp.debug(YoloOutput())

Model: "YOLO_Output"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ output_conv_1               │ (None, 7, 7, 1024)    │  9,442,304 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output_conv_2               │ (None, 4, 4, 1024)    │  9,442,304 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output_conv_3               │ (None, 4, 4, 1024)    │  9,442,304 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output_conv_4               │ (None, 4, 4, 1024)    │  9,442,304 │   Y   │
│ (ConvWithBatchNorm)         │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ flatten (Flatten)           │ (None, 16384)         │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output_fc_1 (Dense)         │ (None, 4096)          │ 67,112,960 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_1 (Dropout)         │ (None, 4096)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output_fc_2 (Dense)         │ (None, 2048)          │  8,390,656 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_2 (Dropout)         │ (None, 2048)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ output_fc_3 (Dense)         │ (None, 1024)          │  2,098,176 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_3 (Dropout)         │ (None, 1024)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ prediction (Dense)          │ (None, 1470)          │  1,506,750 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 116,877,758 (445.85 MB)

 Trainable params: 116,869,566 (445.82 MB)

 Non-trainable params: 8,192 (32.00 KB)

##### YoloV1 

In [16]:
class YoloV1(Sequential):
  """End-to-end YOLO network"""

  def __init__(self, input_shape=INPUT_SHAPE, grid_size=7, num_boxes=2, num_classes=20,
                backbone_config=YOLO_BACKBONE_ARCHITECTURE, name='YOLO_V1'):
    S, B, C = grid_size, num_boxes, num_classes
    yolo_backbone = YoloBackbone(input_shape=input_shape, backbone_config=backbone_config)
    yolo_output = YoloOutput(grid_size=S, num_boxes=B, num_classes=C)
    ### only can adding together, not can store atribute (state) before super() constructor is called
    layers = [keras.Input(shape=INPUT_SHAPE)]
    layers += [yolo_backbone, yolo_output]
    layers += [Reshape((S, S, (5 * B + C)))]
    ### inher customize
    super().__init__(layers=layers, name=name)
    self.a = yolo_backbone
    self.b = yolo_output
    self.S, self.B, self.C = S, B, C 
    
  def call(self, inputs: torch.Tensor, training=False) -> torch.Tensor:
    output = super().call(inputs=inputs, training=training) 
    # torch.tensor(output.numpy()) # convert output to tensor 
    # over(output) # but output is torch.Tensor
    if training:
        return output
    return output 

### YoloLoss 

In [17]:
from keras.losses import Loss 
class YoloLoss(Loss): 
  def __init__(self, coord_c=5, noobj_c=0.5): 
    super().__init__()
    self.COORD = coord_c
    self.NOOBJ = noobj_c 
  
  def expDim(self, val, unsque): 
    return torch.tensor(val, dtype=torch.float32).unsqueeze(unsque)
    
  def computeArea(self, a, b, c, d): 
    return (c-a) * (d-b) 
  def computeIOU(self, cell, cell0): 
    x, y, w, h = cell[..., [0]], cell[..., [1]], cell[..., [2]], cell[..., [3]] 
    x0, y0, w0, h0 = cell0[..., [0]], cell0[..., [1]], cell0[..., [2]], cell0[..., [3]]  
    a, b, c, d = torch.min(x-w, x0-w0), torch.min(y-h, y0-h0), torch.max(x+w, x0+w0), torch.max(y+h, y0+h0)
    return self.computeArea(a, b, c, d) / (self.computeArea(x-w, y-h, x+w, y+h) + self.computeArea(x0-w0, y0-h0, x0+w0, y0+w0) - self.computeArea(a, b, c, d))
  
  def sqrt_sign(self, x): 
    return torch.where(x >= 0, torch.sqrt(x), -torch.sqrt(-x))
  def computeCoorLoss(self, O, mask0, cell, cell0): 
    x, y, w, h = cell[..., [0]], cell[..., [1]], cell[..., [2]], cell[..., [3]] 
    x0, y0, w0, h0 = cell0[..., [0]], cell0[..., [1]], cell0[..., [2]], cell0[..., [3]] 
#     over(mask0) 
    loss1 = torch.sum(O * mask0 * ((x-x0)**2 + (y-y0)**2))
    w, h, w0, h0 = self.sqrt_sign(w), self.sqrt_sign(h), self.sqrt_sign(w0), self.sqrt_sign(h0)  
    loss2 = torch.sum(O * mask0 * ((w-w0)**2 + (h-h0)**2)) 
    return loss1 + loss2 
  
  def call(self, y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor: 
    cell = y_true[..., :4] # (N, S, S, 4)
    O = y_true[..., [4]]
    # O = self.expDim(y_true[..., 4], 3) # (N, S, S, 1) 
    P = y_true[..., 5:] # (N, S, S, C)
    N = y_true.shape[0] 

    cell1 = y_pred[..., :4]
    O1 = y_pred[..., [4]]
    # O1 = self.expDim(y_pred[..., 4], 3) 
    cell2 = y_pred[..., 5:9]
    O2 = y_pred[..., [9]]
    # O2 = self.expDim(y_pred[..., 9], 3) 
    P0 = y_pred[..., 10:]
    
#     print(O, P, P0) 
    over(y_pred)
    print(torch.min(y_pred), torch.max(y_pred))
    classloss = torch.sum(O * (P - P0)**2)  
    print(classloss) 
    
    iou1 = self.computeIOU(cell=cell, cell0=cell1)
    iou2 = self.computeIOU(cell=cell, cell0=cell2) 
    mask1 = torch.where(iou1 > iou2, 1, 0)
    mask2 = torch.where(iou1 < iou2, 1, 0) 
    coorloss = self.computeCoorLoss(O, mask1, cell, cell1) + self.computeCoorLoss(O, mask2, cell, cell2) 
    coorloss *= self.COORD    
    print(coorloss) 
    
    objloss = torch.sum(O * mask1 * (O-O1)**2) + torch.sum(O * mask2 * (O-O2)**2)
    print(objloss)
    
    noobjloss = torch.sum((1-O) * (O-O1)**2 + (1-O) * (O-O2)**2)
    noobjloss *= self.NOOBJ 
    print(noobjloss) 
    return (classloss + coorloss + objloss + noobjloss) / N 
  

In [18]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [19]:
imgs = torch.rand(16, 224, 224, 3).to(DEVICE) 
model = YoloV1()
out = model(imgs) 
over(out) 

__verbose__: over: (<class 'torch.Tensor'>, torch.Size([16, 7, 7, 30]), '80Bytes')


In [20]:
out_true = torch.rand(16, 7, 7, 25).to(DEVICE) 
over(out_true) 
# print(out_true)
loss = YoloLoss()
loss.call(y_pred=out, y_true=out_true) 

__verbose__: over: (<class 'torch.Tensor'>, torch.Size([16, 7, 7, 25]), '80Bytes')
__verbose__: over: (<class 'torch.Tensor'>, torch.Size([16, 7, 7, 30]), '80Bytes')
tensor(-1.6802e-05, device='cuda:0', grad_fn=<MinBackward1>) tensor(1.9366e-05, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(2591.5640, device='cuda:0', grad_fn=<SumBackward0>)
tensor(2464.1858, device='cuda:0', grad_fn=<MulBackward0>)
tensor(146.7066, device='cuda:0', grad_fn=<AddBackward0>)
tensor(64.8907, device='cuda:0', grad_fn=<MulBackward0>)


tensor(329.2092, device='cuda:0', grad_fn=<DivBackward0>)

### DataLoad

In [21]:
import os
from xml.etree import ElementTree
import tensorflow as tf
from tqdm import tqdm
from functools import partial
from keras.preprocessing.image import load_img, img_to_array
from torch.utils.data import Dataset, DataLoader

class_names = ['apple', 'banana', 'orange']

class DataLoad(Dataset): 
  def __init__(self, file_dir, input_shape, grid_size=7) -> None:
    super().__init__ 
    dataframe = self.get_dataframe(file_dir=file_dir)
    self.imgs, self.labels = self.load_dataset(dataframe, input_shape, grid_size) # np.ndarray 
  
  def __len__(self):
    return len(self.imgs)

  def __getitem__(self, idx): #!!! get data 
      x, y = self.imgs[idx], self.labels[idx] # np.ndarray 
      x, y = tf.convert_to_tensor(x), tf.convert_to_tensor(y) # tf.tensor 
      x, y = self._apply_augmentation(x, y, seed=RANDOM_STATE) # tf.tensor 
      # cast type 
      x = torch.tensor(x.numpy(), dtype=torch.float32)  # torch.tensor 
      y = torch.tensor(y.numpy(), dtype=torch.float32)
      # y = torch.tensor(y.numpy(), dtype=torch.float32).unsqueeze(0) over into [N, 1, ...]
      x, y = x.to(DEVICE), y.to(DEVICE) # torch.tensor.device 
#       x, y = self._apply_augmentation(x, y, seed=RANDOM_STATE) # tf.tensor 
      # over(x)
      # over(y) 
      return x, y
  
  
  def get_dataframe(self, file_dir):
    """
    Get the train/val/test dataframe which contains image
    file names and annotations files. If `phase = train',
    return train and val set
    :param file_dir: File directory to create dataframe
    :return file_df: Train or test dataframe
    """

    img_files = [os.path.join(file_dir, img_file) for img_file
                 in sorted(os.listdir(file_dir)) if img_file[-4:] == '.jpg']
    annot_files = [img_file[:-4] + '.xml' for img_file in img_files]

    img_file_series = pd.Series(img_files, name='Image_file')
    annot_file_series = pd.Series(annot_files, name='Annotation_file')
    file_df = pd.DataFrame(pd.concat([img_file_series, annot_file_series], axis=1))

    return file_df

  def prepare_image(self, filename, input_shape):
    """
    Resize image to expected dimension, and opt. apply some random transformation.
    :param filename: File name
    :param input_shape: Shape expected by the model (image will be resize accordingly)
    :return : 3D image array, pixel values from [0., 1.]
    """

    img = img_to_array(load_img(filename, target_size=input_shape)) / 255.

    return img

  def convert_to_xywh(self, bboxes):
    """
    Convert list of (xmin, ymin, xmax, ymax) to
    (x_center, y_center, box_width, box_height)
    :param bboxes: List of bounding boxes, each has 4
    values (xmin, ymin, xmax, ymax)
    :return boxes: List of bounding boxes, each has 4
    values (x_center, y_center, box_width, box_height)
    """

    boxes = list()
    for box in bboxes:
        xmin, ymin, xmax, ymax = box

        # Compute width and height of box
        box_width = xmax - xmin
        box_height = ymax - ymin

        # Compute x, y center
        x_center = int(xmin + (box_width / 2))
        y_center = int(ymin + (box_height / 2))

        boxes.append((x_center, y_center, box_width, box_height))

    return boxes

  def extract_annotation_file(self, filename):
    """
    Extract bounding boxes from an annotation file
    :param filename: Annotation file name
    :return boxes: List of bounding boxes in image, each box has
    4 values (x_center, y_center, box_width, box_height)
    :return classes: List of classes in image
    :return width: Width of image
    :return height: Height of image
    """

    # Load and parse the file
    tree = ElementTree.parse(filename)
    # Get the root of the document
    root = tree.getroot()
    boxes = list()
    classes = list()

    # Extract each bounding box
    for box in root.findall('.//object'):
        cls = class_names.index(box.find('name').text)
        xmin = int(box.find('bndbox/xmin').text)
        ymin = int(box.find('bndbox/ymin').text)
        xmax = int(box.find('bndbox/xmax').text)
        ymax = int(box.find('bndbox/ymax').text)
        coors = (xmin, ymin, xmax, ymax)
        boxes.append(coors)
        classes.append(cls)

    boxes = self.convert_to_xywh(boxes)

    # Get width and height of an image
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)

    # Some annotation files have set width and height by 0,
    # so we need to load image and get it width and height
    if (width == 0) or (height == 0):
        img = load_img(filename[:-4] + '.jpg')
        width, height = img.width, img.height

    return boxes, classes, width, height

  def convert_bboxes_to_tensor(self, bboxes, classes, img_width, img_height, grid_size=7):
    """
    Convert list of bounding boxes to tensor target
    :param bboxes: List of bounding boxes in image, each box has
    4 values (x_center, y_center, box_width, box_height)
    :param classes: List of class in image
    :param img_width: Image's width
    :param img_height: Image's height
    :param grid_size: Grid size
    :return target: Target tensor (grid_size x grid_size x (5 + num_classes))
    """

    num_classes = len(class_names)
    target = np.zeros(shape=(grid_size, grid_size, 5 + num_classes), dtype=np.float32)

    for idx, bbox in enumerate(bboxes):
        x_center, y_center, width, height = bbox

        # Compute size of each cell in grid
        cell_w, cell_h = img_width / grid_size, img_height / grid_size

        # Determine cell i, j of bounding box
        i, j = int(y_center / cell_h), int(x_center / cell_w)

        # Compute value of x_center and y_center in cell
        x, y = (x_center / cell_w) - j, (y_center / cell_h) - i

        # Normalize width and height of bounding box
        w_norm, h_norm = width / img_width, height / img_height

        # Add bounding box to tensor
        # Set x, y, w, h
        target[i, j, :4] += (x, y, w_norm, h_norm)
        # Set obj score
        target[i, j, 4] = 1.
        # Set class dist.
        target[i, j, 5 + classes[idx]] = 1.
    
#     over(target) 
    return target

  def load_dataset(self, dataframe, input_shape, grid_size=7):
    """
    Load img and target tensor
    :param dataframe: Dataframe contains img files and annotation files
    :param input_shape: Shape expected by the model (image will be resize accordingly)
    :param grid_size: Grid size
    :return dataset: Iterable dataset
    """

    imgs, targets = list(), list()

    for _, row in tqdm(dataframe.iterrows()):
        img = self.prepare_image(row.Image_file, input_shape)
        target = self.extract_annotation_file(row.Annotation_file)
        target = self.convert_bboxes_to_tensor(*target, grid_size)
        imgs.append(img)
        targets.append(target)

    imgs = np.array(imgs)
    targets = np.array(targets)
    return imgs, targets 
  
    # dataset = tf.data.Dataset.from_tensor_slices((imgs, targets))
    # return dataset


  def _apply_augmentation(self, image, target, seed=None):
    """
    Apply random brightness and saturation on image
    :param image: Image to augment
    :param target: Target tensor
    :param seed: Seed for random operation
    :return : Processed data
    """

    # Random bright & saturation change
    image = tf.image.random_brightness(image, max_delta=0.1, seed=seed)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5, seed=seed)

    # Keeping pixel values in check
    image = tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

    return image, target


  def load_dataset_from_df(self, dataframe, batch_size=32, num_repeat=None, shuffle=False,
                         input_shape=(448, 448, 3), grid_size=7, augment=False,
                         seed=None):
    """
    Instantiate dataset
    :param dataframe: Dataframe contains img files and annotation files
    :param batch_size: Batch size
    :param num_epochs: Number of epochs (to repeat the iteration - infinite if None)
    :param shuffle: Flag to shuffle the dataset (if True)
    :param input_shape: Shape of the processed image
    :param grid_size: Grid size
    :param augment: Flag to apply some random augmentations to the image
    :param seed: Random seed for operation
    :return : Iterable dataset
    """

    apply_augmentation = partial(self._apply_augmentation, seed=seed)

    dataset = self.load_dataset(dataframe, input_shape, grid_size)
    ### !!!
    dataset = dataset.repeat(num_repeat)
    if shuffle:
        dataset = dataset.shuffle(1000, seed)
    if augment:
        dataset = dataset.map(apply_augmentation, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [22]:
INPUT_SHAPE 

(224, 224, 3)

In [23]:
train_dir = '/kaggle/input/dataset/fruits_dataset/train'
input_shape = INPUT_SHAPE 
grid_size = 7
num_repeat = 2
batch_size = 16
dataload = DataLoad(train_dir, input_shape=INPUT_SHAPE, grid_size=grid_size) 
train_df = dataload.get_dataframe(train_dir)

26it [00:00, 126.25it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
240it [00:02, 83.12it/s] 


In [24]:
over(dataload) 

__verbose__: over: (<class '__main__.DataLoad'>, 'no shape', '48Bytes')


In [25]:
over(dataload[0][0]) 

__verbose__: over: (<class 'torch.Tensor'>, torch.Size([224, 224, 3]), '80Bytes')


In [26]:
over(dataload[0:2][1])

__verbose__: over: (<class 'torch.Tensor'>, torch.Size([2, 7, 7, 8]), '80Bytes')


In [27]:
# Assuming train_dataset is your training dataset
# train_loader = DataLoader(dataset=dataload, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=True, prefetch_factor=2)


In [28]:
# Assuming train_dataset is your training dataset
train_loader = DataLoader(dataset=dataload, batch_size=16, shuffle=True, drop_last=False)

### training and testing 

In [29]:
train_steps_per_epoch = math.ceil(len(train_df) / batch_size)

In [30]:
yolov1 = YoloV1(input_shape=INPUT_SHAPE, num_classes=3)
yolov1.compile(loss=YoloLoss(), optimizer='adam')

In [31]:
out = yolov1(dataload[0:16][0])
over(out) 

__verbose__: over: (<class 'torch.Tensor'>, torch.Size([16, 7, 7, 13]), '80Bytes')


In [32]:
out_true = dataload[0:16][1]
loss_fn = YoloLoss()
loss_vl = loss_fn(y_true=out_true, y_pred=out)

__verbose__: over: (<class 'torch.Tensor'>, torch.Size([16, 7, 7, 13]), '80Bytes')
tensor(-1.8650e-05, device='cuda:0', grad_fn=<MinBackward1>) tensor(2.4156e-05, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(23.0000, device='cuda:0', grad_fn=<SumBackward0>)
tensor(115.0631, device='cuda:0', grad_fn=<MulBackward0>)
tensor(14.0001, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7524e-08, device='cuda:0', grad_fn=<MulBackward0>)


In [33]:
loss_vl

tensor(9.5039, device='cuda:0', grad_fn=<DivBackward0>)

In [1]:
def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []
    
    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        over(x)
        over(y) 
        out = model(x)
        over(out) 
        loss = loss_fn(out, y)
        print(loss)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loop.set_postfix(loss = loss.item())
        
    print(f"Mean loss was {sum(mean_loss) / len(mean_loss)}")

In [5]:
model = YoloV1(input_shape=INPUT_SHAPE, num_classes=3).to(DEVICE)  
optimizer = torch.optim.Adam(
        model.parameters(), lr=2e-5, weight_decay=0
    )
train_fn(train_loader, model, optimizer, loss_fn)

NameError: name 'YoloV1' is not defined

### End 

In [ ]:
print(hist.history['loss']) 

In [ ]:
from keras.losses import MeanSquaredError
l = MeanSquaredError()
l(1, 3.4) 